<a href="https://colab.research.google.com/github/Abhilaksh1498/SoC-20-Chatbot/blob/development/Word_Embedding_deeplearning_ai.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import tensorflow as tf
print(tf.__version__)

2.2.0


We can continue since version >1.x

If it were 1.x we will need to execute
```
tf.enable_eager_execution()
```
Google colab has all tensorflow datasets by default, on a local machine however
you need to execute


```
!pip install -q tensorflow-datasets
```
*Read more about tensorflow datasets here*
https://colab.research.google.com/github/tensorflow/datasets/blob/master/docs/overview.ipynb#scrollTo=byOXYCEJS7S6


In [3]:
import tensorflow_datasets as tfds
imdb, info = tfds.load('imdb_reviews', with_info = True, as_supervised = True) 
# imdb can be thought as list of tuples because of as_supervised argument

Shuffling and writing examples to /root/tensorflow_datasets/imdb_reviews/plain_text/1.0.0.incompleteI0HHP5/imdb_reviews-train.tfrecord


Shuffling and writing examples to /root/tensorflow_datasets/imdb_reviews/plain_text/1.0.0.incompleteI0HHP5/imdb_reviews-test.tfrecord


Shuffling and writing examples to /root/tensorflow_datasets/imdb_reviews/plain_text/1.0.0.incompleteI0HHP5/imdb_reviews-unsupervised.tfrecord


Dataset imdb_reviews downloaded and prepared to /root/tensorflow_datasets/imdb_reviews/plain_text/1.0.0. Subsequent calls will reuse this data.


In [0]:
# imdb has all splits in dictionary format
train_data = imdb['train']
test_data = imdb['test']

More info on splitting/slicing dataset while loading here
https://www.tensorflow.org/datasets/splits

In [0]:
training_sentences = []
training_labels= []

testing_sentences = []
testing_labels = []

for s,l in train_data:
  training_sentences.append(str(s.numpy()))   # tensors are converted to numpy values using .numpy() method
  training_labels.append(l.numpy())           # the text is of type bytes so we can also use decode method

for s,l in test_data:
  testing_sentences.append(str(s.numpy()))   
  testing_labels.append(l.numpy())

In [0]:
for s,l in train_data.take(1):
  print(s)
  print(type(s.numpy().decode('utf-8')))
  print(type(str(s.numpy())))

tf.Tensor(b"This was an absolutely terrible movie. Don't be lured in by Christopher Walken or Michael Ironside. Both are great actors, but this must simply be their worst role in history. Even their great acting could not redeem this movie's ridiculous storyline. This movie is an early nineties US propaganda piece. The most pathetic scenes were those when the Columbian rebels were making their cases for revolutions. Maria Conchita Alonso appeared phony, and her pseudo-love affair with Walken was nothing but a pathetic emotional plug in a movie that was devoid of any real meaning. I am disappointed that there are movies like this, ruining actor's like Christopher Walken's good name. I could barely sit through it.", shape=(), dtype=string)
<class 'str'>
<class 'str'>


In [0]:
vocab_size = 10000
embedding_dim = 16
trunc_type = 'post'
oov_token = 'UNK'
max_length = 100

# While training the labels are expected to be numpy arrays 
import numpy as np
testing_labels = np.array(testing_labels)
training_labels = np.array(training_labels)

In [0]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

tokenizer = Tokenizer(num_words = vocab_size, oov_token = oov_token)
tokenizer.fit_on_texts(training_sentences)
sequences = tokenizer.texts_to_sequences(training_sentences)
training_padded = pad_sequences(sequences, maxlen = max_length, truncating = trunc_type)

In [0]:
testing_sequences = tokenizer.texts_to_sequences(testing_sentences)
testing_padded = pad_sequences(testing_sequences,maxlen=max_length)

In [9]:
word_index = tokenizer.word_index
reverse_word_index = dict([(value, key) for (key, value) in word_index.items()])  # used while saving embeddings

def decode_review(text):
    return ' '.join([reverse_word_index.get(i, '?') for i in text])

print(decode_review(training_padded[3]))
print(training_sentences[3])

? ? ? ? ? ? ? ? ? ? b'this is the kind of film for a snowy sunday afternoon when the rest of the world can go ahead with its own business as you UNK into a big arm chair and UNK for a couple of hours wonderful performances from cher and nicolas cage as always gently row the plot along there are no UNK to cross no dangerous waters just a warm and witty UNK through new york life at its best a family film in every sense and one that deserves the praise it received '
b'This is the kind of film for a snowy Sunday afternoon when the rest of the world can go ahead with its own business as you descend into a big arm-chair and mellow for a couple of hours. Wonderful performances from Cher and Nicolas Cage (as always) gently row the plot along. There are no rapids to cross, no dangerous waters, just a warm and witty paddle through New York life at its best. A family film in every sense and one that deserves the praise it received.'


In [0]:

tk1 = Tokenizer(num_words=6, oov_token='UNK')
tk1.fit_on_texts(['gaddi', 'baddi', 'i', 'am', 'the', 'abhilaksh', 'abhilaksh'])
print(tk1.word_index)
print(tk1.texts_to_sequences(['I am Abhilaksh Maheshwari the god'.lower()]))

{'UNK': 1, 'abhilaksh': 2, 'gaddi': 3, 'baddi': 4, 'i': 5, 'am': 6, 'the': 7}
[[5, 1, 2, 1, 1, 1]]


In [0]:
model = tf.keras.Sequential([
    tf.keras.layers.Embedding(vocab_size, embedding_dim, input_length=max_length),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(6, activation='relu'),
    tf.keras.layers.Dropout(.5),
    tf.keras.layers.Dense(1, activation='sigmoid')
])
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 100, 16)           160000    
_________________________________________________________________
flatten_1 (Flatten)          (None, 1600)              0         
_________________________________________________________________
dense_2 (Dense)              (None, 6)                 9606      
_________________________________________________________________
dropout (Dropout)            (None, 6)                 0         
_________________________________________________________________
dense_3 (Dense)              (None, 1)                 7         
Total params: 169,613
Trainable params: 169,613
Non-trainable params: 0
_________________________________________________________________


In [0]:
num_epochs = 10
model.fit(training_padded, training_labels, epochs=num_epochs, validation_data=(testing_padded, testing_labels), batch_size= 32)

Epoch 1/10
782/782 [==============================] - 3s 4ms/step - loss: 0.0792 - accuracy: 0.9696 - val_loss: 1.0953 - val_accuracy: 0.7952
Epoch 2/10
782/782 [==============================] - 3s 4ms/step - loss: 0.0784 - accuracy: 0.9691 - val_loss: 1.1624 - val_accuracy: 0.8017
Epoch 3/10
782/782 [==============================] - 3s 4ms/step - loss: 0.0832 - accuracy: 0.9666 - val_loss: 1.1820 - val_accuracy: 0.7995
Epoch 4/10
782/782 [==============================] - 3s 4ms/step - loss: 0.0774 - accuracy: 0.9700 - val_loss: 1.2317 - val_accuracy: 0.7990
Epoch 5/10
782/782 [==============================] - 3s 4ms/step - loss: 0.0804 - accuracy: 0.9681 - val_loss: 1.2915 - val_accuracy: 0.7981
Epoch 6/10
782/782 [==============================] - 3s 4ms/step - loss: 0.0825 - accuracy: 0.9669 - val_loss: 1.3333 - val_accuracy: 0.7974
Epoch 7/10
782/782 [==============================] - 3s 4ms/step - loss: 0.0811 - accuracy: 0.9675 - val_loss: 1.4818 - val_accuracy: 0.7931
Epoch 

In [0]:
# Knowing the word embeddings
e = model.get_layer(index=0)
weight_vec = e.get_weights()[0]

In [0]:
print(weight_vec.shape)  #(vocab_size, embedding_dim)
print(weight_vec[0])

word_embeddings = {}
for i in range(1,len(weight_vec)):
  word_embeddings[reverse_word_index[i]] = weight_vec[i]


(10000, 16)
[ 0.04050493 -0.00977446 -0.01973621  0.00213873  0.00932214  0.03109011
  0.02012235 -0.03162124 -0.02921152 -0.00899029 -0.00514809 -0.05028746
  0.01791108 -0.02762359  0.04354888 -0.01028336]


***One thing to note is that if VOCAB_SIZE = 1000 then Tokenizer class will conider words having values 1 to VOCAB_SIZE -1 (999) both inclusive***
It'll consider vocab_size -1 num of words and all other words will be given the oov_token value i.e. 1 (even though they may be present in word_index)

---
*Embedding layer takes vocab_size because it will also encounter 0th index due to padding layer*

3D tensor with shape: (batch_size, input_length, output_dim).


In [0]:
# Visualising the embeddings using Tensorflow Embedding projector
import io
tsv_vectors = io.open('vec.tsv', 'w', encoding = 'utf-8')
tsv_words = io.open('word.tsv', 'w', encoding = 'utf-8')

for word,vector in word_embeddings.items():
  tsv_words.write(word+'\n')
  tsv_vectors.write('\t'.join([str(w) for w in vector])+'\n')

tsv_vectors.close()
tsv_words.close()

# We'll use various RNNs here for Text Classification

In [13]:
model_bidirectional = tf.keras.Sequential([
    tf.keras.layers.Embedding(vocab_size, embedding_dim, input_length=max_length),
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(32)),     # dimensionality of the output space
    tf.keras.layers.Dense(6, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid')
])
model_bidirectional.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_2 (Embedding)      (None, 100, 16)           160000    
_________________________________________________________________
bidirectional_1 (Bidirection (None, 64)                12544     
_________________________________________________________________
dense_2 (Dense)              (None, 6)                 390       
_________________________________________________________________
dense_3 (Dense)              (None, 1)                 7         
Total params: 172,941
Trainable params: 172,941
Non-trainable params: 0
_________________________________________________________________


In [15]:
num_epochs = 10
model_bidirectional.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])
model_bidirectional.fit(training_padded, training_labels, epochs=num_epochs, validation_data=(testing_padded, testing_labels))

Epoch 1/10
782/782 [==============================] - 41s 52ms/step - loss: 0.5038 - accuracy: 0.7453 - val_loss: 0.4176 - val_accuracy: 0.8160
Epoch 2/10
782/782 [==============================] - 40s 51ms/step - loss: 0.3238 - accuracy: 0.8655 - val_loss: 0.3880 - val_accuracy: 0.8207
Epoch 3/10
782/782 [==============================] - 40s 51ms/step - loss: 0.2509 - accuracy: 0.9018 - val_loss: 0.4321 - val_accuracy: 0.8235
Epoch 4/10
782/782 [==============================] - 40s 51ms/step - loss: 0.2035 - accuracy: 0.9214 - val_loss: 0.4534 - val_accuracy: 0.8182
Epoch 5/10
782/782 [==============================] - 40s 51ms/step - loss: 0.1665 - accuracy: 0.9370 - val_loss: 0.7289 - val_accuracy: 0.8023
Epoch 6/10
782/782 [==============================] - 40s 51ms/step - loss: 0.1426 - accuracy: 0.9476 - val_loss: 0.6503 - val_accuracy: 0.8083
Epoch 7/10
782/782 [==============================] - 40s 51ms/step - loss: 0.1135 - accuracy: 0.9595 - val_loss: 0.7958 - val_accuracy:

We'll use a GRU bidirectional model to see if there is any change in epoch training time

In [19]:
model = tf.keras.Sequential([
    tf.keras.layers.Embedding(vocab_size, embedding_dim, input_length=max_length),
    tf.keras.layers.Bidirectional(tf.keras.layers.GRU(32)),
    tf.keras.layers.Dense(6, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid')
])
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])
model.summary()


Model: "sequential_5"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_6 (Embedding)      (None, 100, 16)           160000    
_________________________________________________________________
bidirectional_4 (Bidirection (None, 64)                9600      
_________________________________________________________________
dense_10 (Dense)             (None, 6)                 390       
_________________________________________________________________
dense_11 (Dense)             (None, 1)                 7         
Total params: 169,997
Trainable params: 169,997
Non-trainable params: 0
_________________________________________________________________


In [21]:
num_epochs = 5
history = model.fit(testing_padded, training_labels, epochs=num_epochs, validation_data=(testing_padded, testing_labels))

Epoch 1/5
782/782 [==============================] - 41s 52ms/step - loss: 0.6932 - accuracy: 0.5005 - val_loss: 0.6932 - val_accuracy: 0.5000
Epoch 2/5
782/782 [==============================] - 40s 51ms/step - loss: 0.6929 - accuracy: 0.5156 - val_loss: 0.6934 - val_accuracy: 0.4954
Epoch 3/5
782/782 [==============================] - 40s 51ms/step - loss: 0.6708 - accuracy: 0.5846 - val_loss: 0.7225 - val_accuracy: 0.4898
Epoch 4/5
782/782 [==============================] - 40s 51ms/step - loss: 0.5889 - accuracy: 0.6813 - val_loss: 0.8253 - val_accuracy: 0.4927
Epoch 5/5
782/782 [==============================] - 40s 51ms/step - loss: 0.4831 - accuracy: 0.7575 - val_loss: 1.0699 - val_accuracy: 0.5000
